In [5]:
import os
import pandas as pd

In [6]:
os.listdir('data/tmp/')

['mesh',
 'go',
 '.DS_Store',
 'id_mapper',
 'annotation',
 'kegg',
 'interpro',
 'reactome',
 'msigdb']

### create species table

In [7]:
all_species = {
        'bta':[0,'Bos_taurus','ARS-UCD1.2.107'],
        'cap':[1,'Capra_hircus','ARS1.107'],
        'ovi':[2,'Ovis_aries','Oar_v3.1.107'],
        'gal':[3,'Gallus_gallus','gca000002315v5.GRCg6a.107'],
        'sus':[4,'Sus_scrofa','Sscrofa11.1.107'],
        'equ':[5,'Equus_caballus','EquCab3.0.107'],}  
#
all_species_table = pd.DataFrame([
    ['bta',0,'Bos_taurus','ARS-UCD1.2.107'],
    ['cap',1,'Capra_hircus','ARS1.107'],
    ['ovi',2,'Ovis_aries','Oar_v3.1.107'],
    ['gal', 3,'Gallus_gallus','gca000002315v5.GRCg6a.107'],
    ['sus', 4,'Sus_scrofa','Sscrofa11.1.107'],
    ['equ',5,'Equus_caballus','EquCab3.0.107'],
])
all_species_table.columns = ['name_short','ek_species','name_long','current_gtf']
all_species_table

,name_short,ek_species,name_long,current_gtf
0,bta,0,Bos_taurus,ARS-UCD1.2.107
1,cap,1,Capra_hircus,ARS1.107
2,ovi,2,Ovis_aries,Oar_v3.1.107
3,gal,3,Gallus_gallus,gca000002315v5.GRCg6a.107
4,sus,4,Sus_scrofa,Sscrofa11.1.107
5,equ,5,Equus_caballus,EquCab3.0.107


### id mapper

In [8]:
BASE = '/Users/liulihe95/EnrichKitDB'

In [37]:
import numpy as np
input_path = os.path.join(BASE,'data','tmp','id_mapper')
id_mapper_all_updated = pd.read_csv(os.path.join(input_path,'id_mapper_all_updated.txt'),low_memory=False)
id_mapper_all_updated.pop('Unnamed: 0')
id_mapper_all_updated.fillna('')
id_mapper_all_updated = id_mapper_all_updated.sort_values(by=['species','gene_id'])

id_mapper_all_updated.reset_index(drop=True, inplace=True)
id_mapper_all_updated['ek_gene_id'] = id_mapper_all_updated.index
first_column = id_mapper_all_updated.pop('ek_gene_id')
id_mapper_all_updated.insert(0, 'ek_gene_id', first_column)
id_mapper_all_updated.replace(np.nan,'', regex = True)


,ek_gene_id,species,gene_id,ensembl_symbol,entrez_id,ncbi_symbol,vgnc_id,vgnc_symbol,hgnc_orthologs,human_gene_id,human_entrez_id,hgnc_symbol
0,0,0,ENSBTAG00000000005,GRK3,282136,GRK3,VGNC:53904,GRK3,HGNC:290,ENSG00000100077,157.0,GRK3
1,1,0,ENSBTAG00000000008,KCNJ1,539250,KCNJ1,VGNC:30453,KCNJ1,HGNC:6255,ENSG00000151704,3758.0,KCNJ1
2,2,0,ENSBTAG00000000009,FOXF1,524813,FOXF1,VGNC:29084,FOXF1,HGNC:3809,ENSG00000103241,2294.0,FOXF1
3,3,0,ENSBTAG00000000010,UBL7,511289,UBL7,VGNC:50128,UBL7,HGNC:28221,ENSG00000138629,84993.0,UBL7
4,4,0,ENSBTAG00000000011,TDH,511957,TDH,VGNC:108945,TDH,HGNC:15547,ENSG00000154316,157739,TDH
...,...,...,...,...,...,...,...,...,...,...,...,...
328425,328425,5,,,1.14484e+08,MIR6529,,,,,,
328426,328426,5,,,1.14484e+08,MIR10389,,,,,,
328427,328427,5,,,1.14484e+08,MIR6516,,,,,,
328428,328428,5,,,1.14484e+08,MIR33B,,,,,,


In [35]:
type(id_mapper_all_updated.iloc[328429]['gene_id'])

float

In [29]:
id_mapper_all_updated.species.unique()

array([0, 1, 2, 3, 4, 5])

In [7]:
id_mapper_tiny_no_dup = id_mapper_all_updated[['ek_gene_id','gene_id']].drop_duplicates().dropna()
id_mapper_tiny_no_dup

,ek_gene_id,gene_id
0,0,ENSBTAG00000000005
1,1,ENSBTAG00000000008
2,2,ENSBTAG00000000009
3,3,ENSBTAG00000000010
4,4,ENSBTAG00000000011
...,...,...
300735,300735,ENSSSCG00000063551
300736,300736,ENSSSCG00000063552
300737,300737,ENSSSCG00000063553
300738,300738,ENSSSCG00000063554


### genes

In [8]:
input_path = os.path.join(BASE,'data','tmp','annotation')
all_out = []
for item in os.listdir(input_path):
        if item.startswith('genes_'):
            cur_name_short = item.split('_')[-1].split('.')[0]
            tmp_df = pd.read_csv(os.path.join(input_path,item))
            tmp_df.insert(0, "species", all_species[cur_name_short][0])
            all_out.append(tmp_df)
all_out_df = pd.concat(all_out)
all_out_df = all_out_df.sort_values(by=['species','gene_id'])

all_out_df_tagged = pd.merge(all_out_df, id_mapper_tiny_no_dup, how='left', on='gene_id')
first_column = all_out_df_tagged.pop('ek_gene_id')
all_out_df_tagged.insert(0, 'ek_gene_id', first_column)
genes_all = all_out_df_tagged.drop(columns=['gene_id'])
genes_all

,ek_gene_id,species,seqname,start,end,strand,gene_biotype
0,0,0,17,65389743,65505336,+,protein_coding
1,1,0,29,32214439,32244810,-,protein_coding
2,2,0,18,12338037,12342272,+,protein_coding
3,3,0,21,34209956,34223394,+,protein_coding
4,4,0,8,7950815,7971600,-,protein_coding
...,...,...,...,...,...,...,...
172977,132848,5,19,24659939,24678092,+,lncRNA
172978,132849,5,22,49954554,49956116,+,lncRNA
172979,132850,5,9,43554850,43558334,+,lncRNA
172980,132851,5,5,43781991,43782371,+,protein_coding


### exon

In [9]:
input_path = os.path.join(BASE,'data','tmp','annotation')
all_out = []
for item in os.listdir(input_path):
        if item.startswith('exons_'):
            cur_name_short = item.split('_')[-1].split('.')[0]
            tmp_df = pd.read_csv(os.path.join(input_path,item))
#             tmp_df.insert(0, "species", all_species[cur_name_short][0])
            all_out.append(tmp_df)
all_out_df = pd.concat(all_out)
all_out_df = all_out_df.sort_values(by=['gene_id'])

all_out_df_tagged = pd.merge(all_out_df, id_mapper_tiny_no_dup, how='left', on='gene_id')
first_column = all_out_df_tagged.pop('ek_gene_id')
all_out_df_tagged.insert(0, 'ek_gene_id', first_column)
exons_all = all_out_df_tagged.drop(columns=['gene_id'])
exons_all = exons_all.sort_values(by=['ek_gene_id'])
exons_all

,ek_gene_id,exon_id,start,end,gene_biotype,strand,transcript_id
0,0,ENSBTAE00000496938,65389743,65389903,protein_coding,+,ENSBTAT00000000005
22,0,ENSBTAE00000000019,65480158,65480252,protein_coding,+,ENSBTAT00000000005+ENSBTAT00000068309
21,0,ENSBTAE00000240555,65416282,65416358,protein_coding,+,ENSBTAT00000000005
20,0,ENSBTAE00000000010,65442649,65442722,protein_coding,+,ENSBTAT00000000005+ENSBTAT00000068309
19,0,ENSBTAE00000000011,65454225,65454326,protein_coding,+,ENSBTAT00000000005+ENSBTAT00000068309
...,...,...,...,...,...,...,...
1542252,300736,ENSSSCE00000496980,108776332,108776643,protein_coding,-,ENSSSCT00000092591
1542253,300737,ENSSSCE00000481413,14345924,14351048,lncRNA,+,ENSSSCT00000097588
1542254,300738,ENSSSCE00000493407,47881118,47881348,protein_coding,-,ENSSSCT00000096902
1542255,300739,ENSSSCE00000521169,32510844,32511530,protein_coding,+,ENSSSCT00000103472


### feature

In [10]:
input_path = os.path.join(BASE,'data','tmp','annotation')
all_out = []
for item in os.listdir(input_path):
        if item.startswith('features_'):
            cur_name_short = item.split('_')[-1].split('.')[0]
            tmp_df = pd.read_csv(os.path.join(input_path,item))
#             tmp_df.insert(0, "species", all_species[cur_name_short][0])
            all_out.append(tmp_df)
all_out_df = pd.concat(all_out)
all_out_df = all_out_df.sort_values(by=['gene_id'])

all_out_df_tagged = pd.merge(all_out_df, id_mapper_tiny_no_dup, how='left', on='gene_id')
first_column = all_out_df_tagged.pop('ek_gene_id')
all_out_df_tagged.insert(0, 'ek_gene_id', first_column)
features_all = all_out_df_tagged.drop(columns=['gene_id'])
features_all = features_all.sort_values(by=['ek_gene_id'])

In [11]:
features_all

,ek_gene_id,feature,start,end,gene_biotype,strand
0,0,five_prime_utr,65389743,65389790,protein_coding,+
26,0,CDS,65456261,65456335,protein_coding,+
27,0,CDS,65462772,65462833,protein_coding,+
28,0,CDS,65465537,65465588,protein_coding,+
29,0,stop_codon,65504654,65504656,protein_coding,+
...,...,...,...,...,...,...
3639823,300739,CDS,32510844,32511530,protein_coding,+
3639820,300739,start_codon,32510844,32510846,protein_coding,+
3639821,300739,CDS,32538368,32538457,protein_coding,+
3639822,300739,stop_codon,32538458,32538460,protein_coding,+


### computed feature

In [12]:
input_path = os.path.join(BASE,'data','tmp','annotation')
all_out = []
for item in os.listdir(input_path):
        if item.startswith('computed_feature_'):
            cur_name_short = item.split('_')[-1].split('.')[0]
            tmp_df = pd.read_csv(os.path.join(input_path,item))
#             tmp_df.insert(0, "species", all_species[cur_name_short][0])
            all_out.append(tmp_df)
all_out_df = pd.concat(all_out)
all_out_df = all_out_df.sort_values(by=['gene_id'])

all_out_df_tagged = pd.merge(all_out_df, id_mapper_tiny_no_dup, how='left', on='gene_id')
first_column = all_out_df_tagged.pop('ek_gene_id')
all_out_df_tagged.insert(0, 'ek_gene_id', first_column)
computed_feature_all = all_out_df_tagged.drop(columns=['gene_id'])
computed_feature_all

,ek_gene_id,feature,start,end,gene_biotype,strand
0,0,splice acceptor,65416015,65416065,protein_coding,+
1,0,splice acceptor,65485580,65485630,protein_coding,+
2,0,splice donor,65485739,65485789,protein_coding,+
3,0,intron,65485790,65491986,protein_coding,+
4,0,splice acceptor,65491987,65492037,protein_coding,+
...,...,...,...,...,...,...
3784736,300739,upstream,32500843,32510843,protein_coding,+
3784737,300739,splice donor,32511531,32511581,protein_coding,+
3784738,300739,intron,32511582,32538316,protein_coding,+
3784739,300739,splice acceptor,32538317,32538367,protein_coding,+


### pathway meta

In [14]:
from datetime import datetime
current_time = datetime.now().strftime("%D")
current_time

'10/03/22'

In [15]:
list(id_mapper_all_updated.columns)

['ek_gene_id',
 'species',
 'gene_id',
 'ensembl_symbol',
 'entrez_id',
 'ncbi_symbol',
 'vgnc_id',
 'vgnc_symbol',
 'hgnc_orthologs',
 'human_gene_id',
 'human_entrez_id',
 'hgnc_symbol']

In [16]:
# pathway_meta_id = models.IntegerField(verbose_name='database unit meta_data id', primary_key=True)
# species = models.CharField(verbose_name='current species', max_length=10)
# name = models.CharField(verbose_name='current db unit name', max_length=20)
# id_type = models.CharField(verbose_name='the id type of the current db unit', max_length=50)
# update_time = models.CharField(verbose_name='current db unit update time', max_length=20)

NameError: name 'models' is not defined

In [52]:
all_db_names = ['msigdb', 'go','interpro', 'kegg','mesh','reactome']
all_species_names = list(all_species.keys())
all_db_id_type = {
    'msigdb': 'human_entrez_id',
    'go': 'gene_id',
    'interpro': 'gene_id',
    'kegg': 'entrez_id',
    'mesh': 'entrez_id',
    'reactome':'entrez_id',
}
out = []
for db in all_db_names:
    pathways_dir_path = os.path.join(BASE,'data','tmp', db)
    involve_list = os.listdir(pathways_dir_path)
    if db == 'msigdb':
        tmp_tuple = ['all', db, all_db_id_type[db], current_time]
        out.append(tmp_tuple)
    else:
        for species_involve in involve_list:
            if species_involve.endswith('_involve.txt'):
                tmp_tuple = [species_involve.split('_')[0],db, all_db_id_type[db], current_time]
                out.append(tmp_tuple)             
pathway_meta = pd.DataFrame(out)
pathway_meta.columns = ['species','name','id_type','update_time']
pathway_meta.reset_index(drop=True, inplace=True)
pathway_meta.insert(0, 'pathway_meta_id', pathway_meta.index + 1)
pathway_meta

,pathway_meta_id,species,name,id_type,update_time
0,1,all,msigdb,human_entrez_id,10/03/22
1,2,ovi,go,gene_id,10/03/22
2,3,bta,go,gene_id,10/03/22
3,4,equ,go,gene_id,10/03/22
4,5,gal,go,gene_id,10/03/22
5,6,cap,go,gene_id,10/03/22
6,7,sus,go,gene_id,10/03/22
7,8,ovi,interpro,gene_id,10/03/22
8,9,bta,interpro,gene_id,10/03/22
9,10,equ,interpro,gene_id,10/03/22


### pathway

In [104]:
all_db_names = ['msigdb', 'go','interpro', 'kegg','mesh','reactome']
all_species_names = list(all_species.keys())
all_db_id_type = {
    'msigdb': 'human_entrez_id',
    'go': 'gene_id',
    'interpro': 'gene_id',
    'kegg': 'entrez_id',
    'mesh': 'entrez_id',
    'reactome':'entrez_id',
}
out = []
for db in all_db_names:
    pathways_dir_path = os.path.join(BASE,'data','tmp', db)
    pathway_description_list = os.listdir(pathways_dir_path)        
    if db == 'msigdb':
        for description in pathway_description_list:
            if description.endswith('pathway_list.txt'):
                tmp_df = pd.read_csv(os.path.join(pathways_dir_path, description))
                tmp_df.insert(2,'pathway_meta',1)
                out.append(tmp_df)

        next
    else:
        for description in pathway_description_list:
            if description.endswith('_description.txt'):
                tmp_db = db
                tmp_species = description.split('_')[0]
                if tmp_db == 'mesh' and tmp_species == 'all':
                    tmp_id_list = pathway_meta.loc[pathway_meta.name == db]['pathway_meta_id']
                    for id_mesh in tmp_id_list:
                        tmp_df = pd.read_csv(os.path.join(pathways_dir_path,description))                    
                        tmp_df.insert(2,'pathway_meta',id_mesh)
                        out.append(tmp_df)
                else:  
                    tmp_id = pathway_meta.loc[(pathway_meta.species == tmp_species) & (pathway_meta.name == db)]['pathway_meta_id']
                    tmp_df = pd.read_csv(os.path.join(pathways_dir_path,description))
                    tmp_df.insert(2,'pathway_meta',tmp_id.values[0])
                    out.append(tmp_df)                             
            
all_pathway = pd.concat(out)
all_pathway.reset_index(drop=True, inplace=True)
all_pathway.insert(0, 'ek_pathway_id', all_pathway.index)
all_pathway

,ek_pathway_id,pathway_id,pathway_description,pathway_meta
0,0,M12793,Ensembl 103 genes in cytogenetic band chr1p11,1
1,1,M14892,Ensembl 103 genes in cytogenetic band chr1p12,1
2,2,M6974,Ensembl 103 genes in cytogenetic band chr1p13,1
3,3,M16131,Ensembl 103 genes in cytogenetic band chr1p21,1
4,4,M15893,Ensembl 103 genes in cytogenetic band chr1p22,1
...,...,...,...,...
278137,278137,R-SSC-380612,Metabolism of serotonin,26
278138,278138,R-SSC-380615,Serotonin clearance from the synaptic cleft,26
278139,278139,R-SSC-9617629,Regulation of FOXO transcriptional activity by...,26
278140,278140,R-SSC-9706374,FLT3 signaling through SRC family kinases,26


In [105]:
all_pathway['pathway_meta'].unique()

array([ 1,  6,  2,  5,  4,  3,  7, 12,  8, 11, 10,  9, 13, 15, 18, 14, 19,
       16, 17, 20, 21, 22, 23, 25, 24, 26])

### involve

In [106]:
all_pathway.columns

Index(['ek_pathway_id', 'pathway_id', 'pathway_description', 'pathway_meta'], dtype='object')

In [107]:
pathway_meta.dtypes

pathway_meta_id     int64
species            object
name               object
id_type            object
update_time        object
dtype: object

In [2]:
out = []
id_mapper_all_updated = id_mapper_all_updated.astype(str)
for db in all_db_names:
    pathways_dir_path = os.path.join(BASE,'data','tmp', db)
    pathway_involve_list = os.listdir(pathways_dir_path)        
    if db == 'msigdb':
        for involve in pathway_involve_list:
            if involve.endswith('involve_list.txt'):
                print('currently compose involve list for msigdb')
                tmp_df = pd.read_csv(os.path.join(pathways_dir_path,involve))
                tmp_df = tmp_df.astype({'gene_id': float})
                tmp_df = tmp_df.dropna().astype({'gene_id': int})
                tmp_df = tmp_df.astype({'gene_id': str})
                
                target_pathway_meta = 1
        
                pathway_map_tmp = all_pathway[all_pathway['pathway_meta'] == target_pathway_meta]
        
                tmp_df_2 = pd.merge(tmp_df, pathway_map_tmp[['ek_pathway_id', 'pathway_id']],
                                    how = 'left', 
                                    on='pathway_id')
                out_tmp = tmp_df_2[['gene_id','ek_pathway_id']]
                out.append(out_tmp)
        next
    else:
        for involve in pathway_involve_list:
            if involve.endswith('involve.txt'):
                tmp_db = db
                tmp_species = involve.split('_')[0]
                print('currently compose involve list of ', tmp_db, ' database for species ', tmp_species)
                tmp_df = pd.read_csv(os.path.join(pathways_dir_path,involve)).drop_duplicates()
                tmp_df = tmp_df.astype(str)
                
                id_type = pathway_meta.loc[(pathway_meta.name == tmp_db) & (pathway_meta.species == tmp_species)]['id_type'].values[0]
                
                mapper_tmp  = id_mapper_all_updated[['ek_gene_id',id_type]].dropna().drop_duplicates()
                if id_type == 'entrez_id':
                    mapper_tmp = mapper_tmp.astype({id_type: float})
                    mapper_tmp = mapper_tmp.dropna().astype({id_type: int})
                    mapper_tmp = mapper_tmp.astype({id_type: str})
                    
                tmp_df_1 = pd.merge(tmp_df, mapper_tmp,
                                    how = 'left', 
                                    left_on='gene_id', right_on = id_type).dropna()
                
                target_pathway_meta = pathway_meta.loc[(pathway_meta.name == tmp_db) & (pathway_meta.species == tmp_species)]['pathway_meta_id'].values[0]
                pathway_map_tmp = all_pathway[all_pathway['pathway_meta'] == target_pathway_meta]
                
                tmp_df_2 = pd.merge(tmp_df_1, pathway_map_tmp[['ek_pathway_id', 'pathway_id']],
                                    how = 'left', 
                                    on='pathway_id')#.dropna()
                
                out_tmp = tmp_df_2[['ek_gene_id','ek_pathway_id']]
                out.append(out_tmp)
print('Done!')

NameError: name 'id_mapper_all_updated' is not defined

In [1]:
out

NameError: name 'out' is not defined

In [97]:
tmp_df_2

,gene_id,pathway_id,ek_gene_id,entrez_id,ek_pathway_id
0,100037269,R-SSC-382551,273382,100037269,276640
1,100037269,R-SSC-425397,273382,100037269,276641
2,100037269,R-SSC-425407,273382,100037269,276642
3,100037269,R-SSC-804914,273382,100037269,276643
4,100037275,R-SSC-381426,270594,100037275,276644
...,...,...,...,...,...
22992,808512,R-SSC-6799198,275785,808512,276707
22993,808513,R-SSC-1428517,275787,808513,276686
22994,808513,R-SSC-1430728,275787,808513,276657
22995,808513,R-SSC-163200,275787,808513,276688


In [51]:
pathway_meta

,pathway_meta_id,species,name,id_type,update_time
0,1,all,msigdb,human_entrez_id,10/03/22
1,2,ovi,go,gene_id,10/03/22
2,3,bta,go,gene_id,10/03/22
3,4,equ,go,gene_id,10/03/22
4,5,gal,go,gene_id,10/03/22
5,6,cap,go,gene_id,10/03/22
6,7,sus,go,gene_id,10/03/22
7,8,ovi,interpro,gene_id,10/03/22
8,9,bta,interpro,gene_id,10/03/22
9,10,equ,interpro,gene_id,10/03/22
